# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [232]:
!pip install ultralytics

In [233]:
import cv2
from ultralytics import YOLO, solutions
from ultralytics.utils.plotting import Annotator, colors

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [234]:
track_history = defaultdict(lambda: [])

In [235]:
model = YOLO("/content/YOLO_Model.pt")

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [236]:
cap = cv2.VideoCapture("/content/YOLO_Video.mp4")
assert cap.isOpened(), "Error reading video file"

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [237]:
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [238]:
video_writer = cv2.VideoWriter("object_counting_output.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [239]:
region_points = [(377, 392),(368, 498),(550, 516),(550, 337),(377, 380)],[(1413, 652),(1343, 534),(1210, 580),(1298, 719),(1413,392)]

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [240]:
from collections import defaultdict

In [241]:
counter = solutions.ObjectCounter(
    view_img=True,
    reg_pts=[(377, 392),(368, 498),(550, 516),(550, 337),(377, 380),(1413, 652),(1343, 534),(1210, 580),(1298, 719),(1413,652),(1131, 186),(1065, 186),(1065, 289),(1155, 292),(1131, 186),(794, 840),(717, 833),(714, 901),(791, 898)],
    names=model.names,
    draw_tracks=True,
    line_thickness=2,
    view_in_counts=True,
    count_txt_color = (255,25,25)
)

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Polygon Counter Initiated.


In [242]:
# font
font = cv2.FONT_HERSHEY_SIMPLEX

# org
org = (00, 185)

# fontScale
fontScale = 1

# Red color in BGR
color = (0, 0, 255)

# Line thickness of 2 px
thickness = 2

In [243]:

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    tracks = model.track(im0, persist=True, show=False)
    im0 = counter.start_counting(im0, tracks)
    video_writer.write(im0)
    annotator = Annotator(im0, line_width=2)

    results = model.track(im0, persist=True)


    if results[0].boxes.id is not None and results[0].masks is not None:
        masks = results[0].masks.xy
        track_ids = results[0].boxes.id.int().cpu().tolist()

        for mask, track_id in zip(masks, track_ids):
            color = colors(int(track_id), True)
            txt_color = annotator.get_txt_color(color)
            annotator.seg_bbox(mask=mask, mask_color=(255,0 ,0), label=str(track_id), txt_color=txt_color)

    text = 'abdulmlik_faden'
    image = cv2.putText(im0, text, org, font, fontScale, color, thickness, cv2.LINE_AA, False)
    im0 = counter.start_counting(im0, tracks)
    video_writer.write(im0)


cap.release()
video_writer.release()
cv2.destroyAllWindows()


0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 65.5ms
Speed: 2.5ms preprocess, 65.5ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 buss, 38 cars, 1 truck, 1 van, 41.9ms
Speed: 4.1ms preprocess, 41.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 40.9ms
Speed: 3.4ms preprocess, 40.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 30 cars, 1 truck, 2 vans, 40.8ms
Speed: 2.6ms preprocess, 40.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 40.8ms
Speed: 3.0ms preprocess, 40.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 buss, 29 cars, 1 truck, 1 van, 40.5ms
Speed: 2.9ms preprocess, 40.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 40.0ms
Speed: 3.7ms preprocess, 40.0ms inference, 1.

## Save and Submit Your Work

Submit both the notebook and the output video

In [ ]:
import shutil